In [3]:
import numpy as np
import pandas as pd
from sklearn.utils import resample
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
from os import mkdir
import sys
import pickle



def load_RDS(file_path):
    read_RDS = robjects.r['readRDS']
    return read_RDS(file_path)



def pandas_to_r(df):        
    with localconverter(robjects.default_converter + pandas2ri.converter):
        r_from_pandas_df = robjects.conversion.py2rpy(df)
    return r_from_pandas_df



def r_to_pandas(df):
    with localconverter(robjects.default_converter + pandas2ri.converter):
        pandas_from_r_df = robjects.conversion.rpy2py(df)
    return pandas_from_r_df


In [4]:
path = "/home/colombelli/Documents/datasets/thyroid_log2.rds"

In [5]:
rdf = load_RDS(path)

In [9]:
for name in robjects.r['colnames'](rdf):
    print(name)

DarkCorner
dmr_285
dmr_3
dmr_308
dmr_316
dmr_31a
dmr_6
ebv-miR-BART1-3p
ebv-miR-BART1-5p
ebv-miR-BART10
ebv-miR-BART10*
ebv-miR-BART11-3p
ebv-miR-BART11-5p
ebv-miR-BART12
ebv-miR-BART13
ebv-miR-BART13*
ebv-miR-BART14
ebv-miR-BART14*
ebv-miR-BART15
ebv-miR-BART16
ebv-miR-BART17-3p
ebv-miR-BART17-5p
ebv-miR-BART18-3p
ebv-miR-BART18-5p
ebv-miR-BART19-3p
ebv-miR-BART19-5p
ebv-miR-BART2-3p
ebv-miR-BART2-5p
ebv-miR-BART20-3p
ebv-miR-BART20-5p
ebv-miR-BART3
ebv-miR-BART3*
ebv-miR-BART4
ebv-miR-BART5
ebv-miR-BART6-3p
ebv-miR-BART6-5p
ebv-miR-BART7
ebv-miR-BART7*
ebv-miR-BART8
ebv-miR-BART8*
ebv-miR-BART9
ebv-miR-BART9*
ebv-miR-BHRF1-1
ebv-miR-BHRF1-2
ebv-miR-BHRF1-2*
ebv-miR-BHRF1-3
hcmv-miR-UL112
hcmv-miR-UL148D
hcmv-miR-UL22A
hcmv-miR-UL22A*
hcmv-miR-UL36
hcmv-miR-UL36*
hcmv-miR-UL70-3p
hcmv-miR-UL70-5p
hcmv-miR-US25-1
hcmv-miR-US25-1*
hcmv-miR-US25-2-3p
hcmv-miR-US25-2-5p
hcmv-miR-US33-3p
hcmv-miR-US33-5p
hcmv-miR-US4
hcmv-miR-US5-1
hcmv-miR-US5-2
hiv1-miR-H1
hiv1-miR-N367
hsa-let-7a
hsa-le

In [10]:
pdf = r_to_pandas(rdf)

In [11]:
for col in pdf.columns: 
    print(col) 

DarkCorner
dmr_285
dmr_3
dmr_308
dmr_316
dmr_31a
dmr_6
ebv-miR-BART1-3p
ebv-miR-BART1-5p
ebv-miR-BART10
ebv-miR-BART10*
ebv-miR-BART11-3p
ebv-miR-BART11-5p
ebv-miR-BART12
ebv-miR-BART13
ebv-miR-BART13*
ebv-miR-BART14
ebv-miR-BART14*
ebv-miR-BART15
ebv-miR-BART16
ebv-miR-BART17-3p
ebv-miR-BART17-5p
ebv-miR-BART18-3p
ebv-miR-BART18-5p
ebv-miR-BART19-3p
ebv-miR-BART19-5p
ebv-miR-BART2-3p
ebv-miR-BART2-5p
ebv-miR-BART20-3p
ebv-miR-BART20-5p
ebv-miR-BART3
ebv-miR-BART3*
ebv-miR-BART4
ebv-miR-BART5
ebv-miR-BART6-3p
ebv-miR-BART6-5p
ebv-miR-BART7
ebv-miR-BART7*
ebv-miR-BART8
ebv-miR-BART8*
ebv-miR-BART9
ebv-miR-BART9*
ebv-miR-BHRF1-1
ebv-miR-BHRF1-2
ebv-miR-BHRF1-2*
ebv-miR-BHRF1-3
hcmv-miR-UL112
hcmv-miR-UL148D
hcmv-miR-UL22A
hcmv-miR-UL22A*
hcmv-miR-UL36
hcmv-miR-UL36*
hcmv-miR-UL70-3p
hcmv-miR-UL70-5p
hcmv-miR-US25-1
hcmv-miR-US25-1*
hcmv-miR-US25-2-3p
hcmv-miR-US25-2-5p
hcmv-miR-US33-3p
hcmv-miR-US33-5p
hcmv-miR-US4
hcmv-miR-US5-1
hcmv-miR-US5-2
hiv1-miR-H1
hiv1-miR-N367
hsa-let-7a
hsa-le

In [21]:
# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, "/home/colombelli/Documents/bioinformatics-ml/EnsembleSelector-v2.0")

from DataManager import DataManager
from EFS import EFS
from Evaluator import Evaluator
import rpy2.robjects.packages as rpackages

In [22]:
seed = 42
num_bootstraps = 5
num_folds = 3

fs_methods = [
    ("gain-ratio", "r", "gr")
]

dataset_path = "/home/colombelli/Documents/datasets/thyroid_log2.rds"
results_path = "/home/colombelli/Documents/bioinformatics-ml/Experiments/debug-res/"

rpackages.importr('FSelectorRcpp')

aggregator = "mean"

In [23]:
dm = DataManager(results_path, dataset_path, num_bootstraps, num_folds, seed)

Creating results directory...


In [26]:
for col in dm.pd_df.columns:
    print(col)

DarkCorner
dmr_285
dmr_3
dmr_308
dmr_316
dmr_31a
dmr_6
ebv-miR-BART1-3p
ebv-miR-BART1-5p
ebv-miR-BART10
ebv-miR-BART10*
ebv-miR-BART11-3p
ebv-miR-BART11-5p
ebv-miR-BART12
ebv-miR-BART13
ebv-miR-BART13*
ebv-miR-BART14
ebv-miR-BART14*
ebv-miR-BART15
ebv-miR-BART16
ebv-miR-BART17-3p
ebv-miR-BART17-5p
ebv-miR-BART18-3p
ebv-miR-BART18-5p
ebv-miR-BART19-3p
ebv-miR-BART19-5p
ebv-miR-BART2-3p
ebv-miR-BART2-5p
ebv-miR-BART20-3p
ebv-miR-BART20-5p
ebv-miR-BART3
ebv-miR-BART3*
ebv-miR-BART4
ebv-miR-BART5
ebv-miR-BART6-3p
ebv-miR-BART6-5p
ebv-miR-BART7
ebv-miR-BART7*
ebv-miR-BART8
ebv-miR-BART8*
ebv-miR-BART9
ebv-miR-BART9*
ebv-miR-BHRF1-1
ebv-miR-BHRF1-2
ebv-miR-BHRF1-2*
ebv-miR-BHRF1-3
hcmv-miR-UL112
hcmv-miR-UL148D
hcmv-miR-UL22A
hcmv-miR-UL22A*
hcmv-miR-UL36
hcmv-miR-UL36*
hcmv-miR-UL70-3p
hcmv-miR-UL70-5p
hcmv-miR-US25-1
hcmv-miR-US25-1*
hcmv-miR-US25-2-3p
hcmv-miR-US25-2-5p
hcmv-miR-US33-3p
hcmv-miR-US33-5p
hcmv-miR-US4
hcmv-miR-US5-1
hcmv-miR-US5-2
hiv1-miR-H1
hiv1-miR-N367
hsa-let-7a
hsa-le

In [68]:
from Selector import PySelector, RSelector
from Aggregator import Aggregator
from DataManager import DataManager
import time

class EFS:
    
    # fs_methods: a tuple (script name, language which the script was written, .rds output name)
    def __init__(self, data_manager:DataManager, fs_methods, first_aggregator, second_aggregator):

        self.dm = data_manager
        self.fs_methods = self.__generate_fselectors_object(fs_methods)
        self.fst_aggregator = Aggregator(first_aggregator, self.dm)
        self.snd_aggregator = Aggregator(second_aggregator, self.dm)


        
    def __generate_fselectors_object(self, methods):
        
        fs_methods = []
        for script, language, rds_name in methods:
            if language == "python":
                fs_methods.append(
                    PySelector(rds_name, script)
                )
            elif language == "r":
                fs_methods.append(
                    RSelector(rds_name, script)
                )

        return fs_methods




    def end_time_and_print(self, start):
        end = time.time()
        hours, rem = divmod(end-start, 3600)
        minutes, seconds = divmod(rem, 60)
        print("\nTime taken:")
        print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
        print("\n")
        return


    def select_features(self):

        for i in range(self.dm.num_folds):
            print("\n\n################# Fold iteration:", i+1, "#################")
            self.dm.current_fold_iteration = i
            self.dm.update_bootstraps()

            self.snd_layer_rankings = []
            for j, (bootstrap, _) in enumerate(self.dm.current_bootstraps):
                print("\n\nBootstrap: ", j+1, "\n")
                start = time.time()
                output_path = self.dm.get_output_path(i, j)
                bootstrap_data = self.dm.pd_df.loc[bootstrap]

                self.fst_layer_rankings = []
                for fs_method in self.fs_methods:   
                    print("")
                    bootstrap_data = bootstrap_data.rename({"hsa-miR-375": "aa:.,-=)(*&%$#@!çãõáÀ><;_ []{}|\/?", 
                                           'hsa-miR-376c': "bb:.,-=)(*&%$#@!çãõáÀ><;_ []{}|\/?"}, axis='columns')
                    #self.fst_layer_rankings.append(
                        #fs_method.select(bootstrap_data, output_path)
                    #)
                    print("after features selecting:\n", fs_method.select(bootstrap_data, output_path))
                    input()
                
                self.fs_aggregation = self.fst_aggregator.aggregate(self.fst_layer_rankings)
                self.dm.save_aggregated_ranking(self.fs_aggregation, output_path)
                self.snd_layer_rankings.append(self.fs_aggregation)
                self.end_time_and_print(start)
            
            output_path = self.dm.get_output_path(fold_iteration=i)
            final_ranking = self.snd_aggregator.aggregate(self.snd_layer_rankings)
            self.dm.save_aggregated_ranking(final_ranking, output_path)

In [69]:
ensemble = EFS(dm, fs_methods, aggregator, aggregator)
ensemble.select_features()



################# Fold iteration: 1 #################


Bootstrap:  1 


[1] "Calculating Information Gain Ratio..."
[1] "Processing output..."
[1] "Saving ranking..."
[1] "Ranking saved on:"
[1] "/home/colombelli/Documents/bioinformatics-ml/Experiments/debug-res/fold_1/bootstrap_1/gr.rds"
after features selecting:
                                     rank
hsa.miR.153                            1
aa..............çãõáÀ..._.........     2
hsa.miR.221                            3
hsa.miR.10a                            4
hsa.miR.335                            5
...                                  ...
NC2_00092197                         813
NC2_00106057                         814
NC2_00122731                         815
NegativeControl                      816
SCorner3                             817

[817 rows x 1 columns]


KeyboardInterrupt: 

In [35]:
ensemble.fst_layer_rankings

[                 rank
 hsa.miR.199b.5p     1
 hsa.miR.218         2
 hsa.miR.375         3
 hsa.miR.130a        4
 hsa.miR.153         5
 ...               ...
 NC2_00092197      813
 NC2_00106057      814
 NC2_00122731      815
 NegativeControl   816
 SCorner3          817
 
 [817 rows x 1 columns]]

In [46]:
pdf.rename({'DarkCorner': '*'}, axis='columns')

,*,dmr_285,dmr_3,dmr_308,dmr_316,dmr_31a,dmr_6,ebv-miR-BART1-3p,ebv-miR-BART1-5p,ebv-miR-BART10,...,mr_1,NC1_00000197,NC1_00000215,NC2_00079215,NC2_00092197,NC2_00106057,NC2_00122731,NegativeControl,SCorner3,class
GSM1002105,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,...,12.543404,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,1
GSM1002106,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,...,11.768829,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0
GSM1002107,0.137504,3.182575,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,...,12.487421,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,1
GSM1002108,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,...,10.396037,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0
GSM1002109,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,...,11.317543,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1002180,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,...,10.689570,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0
GSM1002181,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,...,10.474122,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,1
GSM1002182,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,...,10.057965,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0
GSM1002183,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,...,10.121378,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,0.137504,1


In [74]:
import urllib.parse

def alnum_encode(text):
    return urllib.parse.quote(text, safe='')\
        .replace('-', '%2d').replace('.', '%2e').replace('_', '%5f')\
        .replace('%', '_')

def alnum_decode(underscore_encoded):
    return urllib.parse.unquote(underscore_encoded.replace('_','%'), errors='strict')

In [75]:
def encode_df(df):

    for attribute in df.columns:
        enc_attribute = alnum_encode(attribute)
        df = df.rename({attribute: enc_attribute}, axis='columns')
        
    return df

In [ ]:
def decode_df(df):

    for attribute in df.columns:
        dec_attribute = alnum_decode(attribute)
        df = df.rename({attribute: dec_attribute}, axis='columns')
        
    return df

In [79]:
for c in encode_df(pdf).columns:
    print(c)

DarkCorner
dmr_5f285
dmr_5f3
dmr_5f308
dmr_5f316
dmr_5f31a
dmr_5f6
ebv_2dmiR_2dBART1_2d3p
ebv_2dmiR_2dBART1_2d5p
ebv_2dmiR_2dBART10
ebv_2dmiR_2dBART10_2A
ebv_2dmiR_2dBART11_2d3p
ebv_2dmiR_2dBART11_2d5p
ebv_2dmiR_2dBART12
ebv_2dmiR_2dBART13
ebv_2dmiR_2dBART13_2A
ebv_2dmiR_2dBART14
ebv_2dmiR_2dBART14_2A
ebv_2dmiR_2dBART15
ebv_2dmiR_2dBART16
ebv_2dmiR_2dBART17_2d3p
ebv_2dmiR_2dBART17_2d5p
ebv_2dmiR_2dBART18_2d3p
ebv_2dmiR_2dBART18_2d5p
ebv_2dmiR_2dBART19_2d3p
ebv_2dmiR_2dBART19_2d5p
ebv_2dmiR_2dBART2_2d3p
ebv_2dmiR_2dBART2_2d5p
ebv_2dmiR_2dBART20_2d3p
ebv_2dmiR_2dBART20_2d5p
ebv_2dmiR_2dBART3
ebv_2dmiR_2dBART3_2A
ebv_2dmiR_2dBART4
ebv_2dmiR_2dBART5
ebv_2dmiR_2dBART6_2d3p
ebv_2dmiR_2dBART6_2d5p
ebv_2dmiR_2dBART7
ebv_2dmiR_2dBART7_2A
ebv_2dmiR_2dBART8
ebv_2dmiR_2dBART8_2A
ebv_2dmiR_2dBART9
ebv_2dmiR_2dBART9_2A
ebv_2dmiR_2dBHRF1_2d1
ebv_2dmiR_2dBHRF1_2d2
ebv_2dmiR_2dBHRF1_2d2_2A
ebv_2dmiR_2dBHRF1_2d3
hcmv_2dmiR_2dUL112
hcmv_2dmiR_2dUL148D
hcmv_2dmiR_2dUL22A
hcmv_2dmiR_2dUL22A_2A
hcmv_2dmiR_2